In [2]:
import json
import os
import numpy
import pandas
import pyarrow
import sys

from datetime import date
from os import path
from dotenv import load_dotenv
from sqlalchemy import create_engine

# Steps to install
# 1. pip install sqlalchemy-bigquery google-cloud-bigquery-storage pyarrow
# 2. Copy the credentials file to wherever you set BIGQUERY_CREDENTIALS_PATH to

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')

In [3]:
sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
import create_queries
%load_ext autoreload
%autoreload 2

In [56]:
COMPANY_NAME = 'Royal Apothecary'
COMPANY_IDENTIFIER = 'RA'
TRANSFER_PACKAGES_START_DATE = '2020-01-01'
SALES_TRANSACTIONS_START_DATE = '2020-01-01'
INVENTORY_DATES = [
    '09/30/2020',
    '10/31/2020',
    '11/30/2020',
    '12/31/2020',
    '01/31/2021',
    '02/28/2021',
    '03/31/2021',
    '04/30/2021',
    '05/31/2021',
    '06/30/2021',
    '07/31/2021',
    '08/31/2021',
    '09/30/2021',
]
ANALYSIS_PARAMS = {
    'sold_threshold': 1.0
}
TODAY_DATE = date.today().strftime('%m/%d/%Y')
INVENTORY_DATES.append(TODAY_DATE)
print('Today is {}'.format(TODAY_DATE))

Today is 10/23/2021


In [5]:
# Download packages, sales transactions, incoming / outgoing tranfers

company_incoming_transfer_packages_query = create_queries.create_company_incoming_transfer_packages_query(COMPANY_IDENTIFIER, TRANSFER_PACKAGES_START_DATE)
company_outgoing_transfer_packages_query = create_queries.create_company_outgoing_transfer_packages_query(COMPANY_IDENTIFIER, TRANSFER_PACKAGES_START_DATE)
company_sales_transactions_query = create_queries.create_company_sales_transactions_query(COMPANY_IDENTIFIER, SALES_TRANSACTIONS_START_DATE)
company_inventory_packages_query = create_queries.create_company_inventory_packages_query(COMPANY_IDENTIFIER)

engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))
company_incoming_transfer_packages_dataframe = pandas.read_sql_query(company_incoming_transfer_packages_query, engine)
company_outgoing_transfer_packages_dataframe = pandas.read_sql_query(company_outgoing_transfer_packages_query, engine)
company_sales_transactions_dataframe = pandas.read_sql_query(company_sales_transactions_query, engine)
company_inventory_packages_dataframe = pandas.read_sql_query(company_inventory_packages_query, engine)

In [17]:
sys.path.append(path.realpath(path.join(os.getcwd(), "../../scripts/analysis")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))

from util import active_inventory_util as util

In [7]:
d = util.Download()
d.download_dataframes(
    incoming_transfer_packages_dataframe=company_incoming_transfer_packages_dataframe,
    outgoing_transfer_packages_dataframe=company_outgoing_transfer_packages_dataframe,
    sales_transactions_dataframe=company_sales_transactions_dataframe,
)

In [57]:
# Check for how many packages are excluded from analysis,
# seeing less then 2% is really good
q = util.Query()
q.inventory_dates = INVENTORY_DATES
q.company_name = COMPANY_NAME

id_to_history = util.get_histories(d)
util.print_counts(id_to_history)
util.create_inventory_xlsx(id_to_history, q, params=ANALYSIS_PARAMS)

Only outgoing: 7
Only incoming: 1525
In and out: 8
In and sold at least once 4058
In and sold many times 3928
Total pkgs: 5634
Wrote result to out/Royal Apothecary_inventory_by_month.xls
Excluded 84 / 5634 packages from consideration (1.49%)
  MANY_INCOMING: 40 times
  MISSING_INCOMING: 44 times


In [58]:
date_to_inventory_packages_dataframe = {}

for inventory_date in INVENTORY_DATES:
    computed_inventory_package_records = util.create_inventory_dataframe_by_date(
        id_to_history, inventory_date, params=ANALYSIS_PARAMS)
    computed_inventory_packages_dataframe = pandas.DataFrame(
        computed_inventory_package_records,
        columns=[
            'package_id',
            'arrived_ate',
            'product_category_name',
            'product_name',
            'quantity',
            'sold_date',
        ]
    )
    date_to_inventory_packages_dataframe[inventory_date] = computed_inventory_packages_dataframe

In [59]:
from_packages_inventory_dataframe = company_inventory_packages_dataframe[[
    'package_id',
    'packaged_date',
    'product_category_name',
    'product_name',
    'quantity',
]].sort_values('package_id')

In [61]:
# For debugging individual package histories
import importlib
importlib.reload(util)

PACKAGE_IDS = [
  '10533612',
  '7935059',
  '10107769',
  '5941782',
]
PACKAGE_IDS = []

util.analyze_specific_package_histories(d, PACKAGE_IDS, params=ANALYSIS_PARAMS)

In [63]:
import importlib
importlib.reload(util)

util.compare_inventory_dataframes(
    computed=date_to_inventory_packages_dataframe[TODAY_DATE],
    actual=from_packages_inventory_dataframe
)

Pct of inventory matching: 77.13%
Accuracy of quantities: 93.94%

Avg quantity delta: 0.29
Avg quantity: 4.81

Num matching packages: 2536
Num actual packages not computed: 752
Num computed packages not in actual: 854

Missing package IDs first 100
7935059
5941782
7610313
2921027
17470279
8443174
3804457
17832731
4971146
6458448
9195941
5409709
6302167
3584073
5492314
16037981
4416970
5561088
9259997
5516915
3472743
6674853
5754882
3578610
3584075
14673166
6250151
3066730
6255159
3066731
8777407
4624944
2641237
5409704
7486661
8777406
8120990
15913790
5092922
6081160
17324734
8749812
9058396
5981386
5852038
4218709
17470285
8702455
3575532
4548518
5006177
7471924
6072879
8908151
5435774
8749807
8367249
8749688
8367259
6712611
5435779
6331129
3454684
6271700
8183656
3472734
8749810
6525636
6197478
3066727
7589770
8224647
3067625
3578615
8269527
8993784
6775605
4426995
3066728
8794715
8645831
2632826
3586693
3962910
7675490
3685940
6081161
7881827
6318543
8227165
6087373
6192867
9204146


In [ ]:
#### Unused, previously used code



####

In [ ]:
# Get dataframe TODAYS_DATE

today_inventory_package_records = util.create_inventory_dataframe_by_date(id_to_history, TODAY_DATE)
today_inventory_packages_dataframe = pandas.DataFrame(
    today_inventory_package_records,
    columns=[
        'package_id',
        'arrived_date',
        'product_category_name',
        'product_name',
        'current_quantity',
        'sold_date',
    ]
)
today_inventory_packages_dataframe.sort_values('package_id')

In [ ]:
# Some extra analysis on inventory


In [ ]:
incoming_transfer_packages_dataframe = company_incoming_transfer_packages_dataframe[[
#     'date_type',
#     'transfer_row_id',
#     'delivery_row_id',
#     'package_row_id',
    'delivery_type',
    'manifest_number',
    'created_date',
    'received_datetime',
    'shipper_facility_license_number',
    'shipper_facility_name',
    'recipient_facility_license_number',
    'recipient_facility_name',
    'shipment_type_name',
    'shipment_transaction_type',
    'package_id',
    'package_label',
    'type',
    'product_category_name',
    'product_name',
    'shipper_wholesale_price',
    'shipped_quantity',
    'package_lab_results_status',
    'shipment_package_state',
    'is_testing_sample',
    'is_trade_sample'
]]
incoming_transfer_packages_dataframe

In [ ]:

import math

for inventory_date, inventory_packages_dataframe in date_to_inventory_packages_dataframe.items():
    print(inventory_date)
    print(f'# of packages in inventory: {len(inventory_packages_dataframe.index)}')

    inventory_with_incoming_transfer_packages_dataframe = inventory_packages_dataframe \
        .astype({'package_id': 'int64'}) \
        .merge(
            incoming_transfer_packages_dataframe.astype({'package_id': 'int64'}),
            on='package_id',
            how='inner',
            suffixes=('_l', '_r')
        )
    #     print(f'# of packages in inventory with incoming package: {len(inventory_with_incoming_transfer_packages_dataframe.index)}')

    inventory_with_cost_records = inventory_with_incoming_transfer_packages_dataframe.to_dict('record')

    total_valuation_cost = 0

    for inventory_with_cost_record in inventory_with_cost_records:
        incoming_shipped_price = inventory_with_cost_record['shipper_wholesale_price']
        if math.isnan(incoming_shipped_price):
            incoming_shipped_price = 0
        incoming_shipped_quantity = inventory_with_cost_record['shipped_quantity']
        current_quantity = inventory_with_cost_record['Current Quantity']
        total_valuation_cost += float(current_quantity) * (incoming_shipped_price / incoming_shipped_quantity)

    print(f'Inventory valuation (based on COST): ${total_valuation_cost}')
    print('')

In [ ]:
company_inventory_packages_dataframe
company_sales_transactions_dataframe
company_outgoing_transfer_packages_dataframe
company_incoming_transfer_packages_dataframe